In [13]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import sqlite3

In [14]:
# J'ouvre ma session request
s = requests.Session()
#Je crée une définition qui va me permettre de créer ma soup avec un url
def get_soup(url):
    response = s.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    return soup

In [15]:
soup = get_soup('https://scramazon.rbean.io/')

In [5]:
main_cat = soup.find_all('h3',{'class':'mb-2'})

In [6]:
main_cat = []
main_cat_url = []
for i in soup.find_all('h3'):
    try:
        main_cat.append(i.find('a',{'class':'text-dark'}).text)
        main_cat_url.append(i.find('a',{'class':'text-dark'},href = True)['href'])
    except:
        continue

In [7]:
df_main = pd.DataFrame()
df_main['Name'] = main_cat
df_main['Slug'] = main_cat_url

In [8]:
df_main

,Name,Slug
0,Toys & Games,/categories?slug=toys-games-0
1,Automotive,/categories?slug=automotive-0


In [30]:
sub_cat = []
main_id = []

for count,slug in enumerate(main_cat_url):
    sub_cats = []
    soup = get_soup('https://scramazon.rbean.io' + slug)
    sub_cats.extend(soup.find_all('a',{'class':'text-truncate text-dark'}))
    while True:
        if soup.find_all('a',{'class':'page-link'})[-1].text == 'Next':
            next_url = soup.find_all('a',{'class':'page-link'})[-1]['href']
            soup  = get_soup('https://scramazon.rbean.io/categories' + next_url)
            sub_cats.extend(soup.find_all('a',{'class':'text-truncate text-dark'}))

        else:
            break
    for i in sub_cats:
        sub_cat.append(i)
        main_id.append(count)
sub_cat_url = [i['href'] for i in sub_cat]
sub_cat = [i.text for i in sub_cat]

In [31]:
df_subcat = pd.DataFrame()
df_subcat['Name'] = sub_cat
df_subcat['Slug'] = sub_cat_url
df_subcat['MainCategoryId'] = main_id

In [41]:
product = []
product_url = []
sub_cat_id = []
for count,slug in enumerate(sub_cat_url):
    products = []
    soup = get_soup('https://scramazon.rbean.io'+slug)
    products.extend(soup.find_all('h5',{'class':'card-title'}))
    product_url.extend(soup.find_all('a',{'class':'btn btn-primary mt-auto mx-auto'}))
    while True:
        try:
            if soup.find_all('a',{'class':'page-link'})[-1].text == 'Next':
                next_url = soup.find_all('a',{'class':'page-link'})[-1]['href']
                soup  = get_soup('https://scramazon.rbean.io/categories' + next_url)
                products.extend(soup.find_all('h5',{'class':'card-title'}))
                product_url.extend(soup.find_all('a',{'class':'btn btn-primary mt-auto mx-auto'}))
            else:
                break
        except:
            break
    for i in products:
        product.append(i)
        sub_cat_id.append(count)
product = [i.text for i in product]
product_url = [i['href'] for i in product_url]

In [42]:
df_product = pd.DataFrame()
df_product['Name'] = product
df_product['Slug'] = product_url
df_product['SubCategoryId'] = sub_cat_id

In [44]:
df_product_no_dup = df_product.drop_duplicates(subset=['Slug']).reset_index().drop(columns='index')

In [45]:
id_p = []
id_sc = []
for count,slug in enumerate(df_product_no_dup['Slug']):
    for id_s in df_product[df_product['Slug'] == slug]['SubCategoryId']:
        id_p.append(count)
        id_sc.append(id_s)
    

In [36]:
comment = []
note = []
prod_id = []
for count,slug in enumerate(df_product['Slug']):
    soup = get_soup('https://scramazon.rbean.io'+slug)
    for i in soup.find_all('div',{'class':'d-flex flex-column border bg-light mb-2 rounded-lg p-3'}):
            note.append(len(i.find_all('svg',{'class':'fill-star'})))
            comment.append(i.find('p').text.strip())
            prod_id.append(count)

In [37]:
df_note = pd.DataFrame()
df_note['Value'] = note
df_note['Comment'] = comment
df_note['ProductId'] = prod_id

In [48]:
df_catprod = pd.DataFrame()
df_catprod['SubCategoryId'] = id_sc
df_catprod['ProductId'] = id_p

In [51]:
df_catprod_no_dup = df_catprod.drop_duplicates()

In [52]:
import sqlite3

conn = sqlite3.connect('scram.db')
c = conn.cursor()

c.execute(''' 
DROP TABLE IF EXISTS Note;
          ''')
c.execute(''' 
DROP TABLE IF EXISTS CategoryProduct;
          ''')
c.execute(''' 
DROP TABLE IF EXISTS Product;
          ''')
c.execute(''' 
DROP TABLE IF EXISTS SubCategory;
          ''')
c.execute(''' 
DROP TABLE IF EXISTS MainCategory;
          ''')

c.execute('CREATE TABLE IF NOT EXISTS MainCategory (ID INT PRIMARY KEY,Name STRING NOT NULL, Slug NOT NULL)')
conn.commit()

df_main.to_sql('MainCategory', conn, if_exists='replace', index = True, index_label ='ID')



c.execute('CREATE TABLE IF NOT EXISTS SubCategory (ID INT PRIMARY KEY NOT NULL,Name STRING NOT NULL, Slug NOT NULL, MainCategoryID,FOREIGN KEY(MainCategoryID) REFERENCES MainCategory(ID))')
conn.commit()

df_subcat.to_sql('SubCategory', conn, if_exists='replace', index = True, index_label ='ID')


    
c.execute('CREATE TABLE IF NOT EXISTS Product (ID INT PRIMARY KEY,Name STRING NOT NULL, Slug NOT NULL)')
conn.commit()

df_product_no_dup.to_sql('Product', conn, if_exists='replace', index = True, index_label ='ID')
 

    
c.execute('CREATE TABLE IF NOT EXISTS CategoryProduct (SubCategoryId INT NOT NULL, ProductId INT NOT NULL,FOREIGN KEY(SubCategoryId) REFERENCES SubCategory(ID),FOREIGN KEY(ProductId) REFERENCES Product(ID))')
conn.commit()

df_catprod_no_dup.to_sql('CategoryProduct', conn, if_exists='replace', index = False)
 
c.execute('CREATE TABLE IF NOT EXISTS Note (ID INT PRIMARY KEY, Value INT NOT NULL,Comment STRING,ProductId,FOREIGN KEY(ProductId) REFERENCES Product(ID))')
conn.commit()

df_note.to_sql('Note',conn,if_exists='replace',index=True, index_label ='ID')

c.close()
conn.close()